In [1]:
import pyspark.sql
import pyspark
'''
conf = pyspark.SparkConf()
conf.setAppName("Pandas_UDF") \
    .setMaster('spark://10.64.22.215:7077') \
    .set('spark.jars.packages','org.diana-hep:spark-root_2.11:0.1.16,org.diana-hep:histogrammar-sparksql_2.11:1.0.4') \
    .set('spark.driver.extraClassPath','/opt/hadoop/share/hadoop/common/lib/EOSfs.jar') \
    .set('spark.executor.extraClassPath','/opt/hadoop/share/hadoop/common/lib/EOSfs.jar') \
    .set('spark.sql.caseSensitive','true') \
    .set('spark.serializer','org.apache.spark.serializer.KryoSerializer') \
    .set("spark.cores.max", "10") \
    .set('spark.executor.instances', '5') \
    .set('spark.executor.cores','2') \
    .set('spark.executor.memory','2g') 

#sc = pyspark.SparkContext(conf=conf)
'''

session = pyspark.sql.SparkSession.builder \
    .master('spark://10.64.22.215:7077') \
    .appName('Pandas_UDF_Example') \
    .config('spark.jars.packages','org.diana-hep:spark-root_2.11:0.1.16,org.diana-hep:histogrammar-sparksql_2.11:1.0.4') \
    .config('spark.driver.extraClassPath','/opt/hadoop/share/hadoop/common/lib/EOSfs.jar') \
    .config('spark.executor.extraClassPath','/opt/hadoop/share/hadoop/common/lib/EOSfs.jar') \
    .config('spark.sql.caseSensitive','true') \
    .config('spark.serializer','org.apache.spark.serializer.KryoSerializer') \
    .config("spark.cores.max", "10") \
    .config('spark.executor.instances', '5') \
    .config('spark.executor.cores','2') \
    .config('spark.executor.memory','2g') \
    .getOrCreate()
    
sqlContext = session

print 'Spark version: ',sqlContext.version
print 'SparkSQL sesssion created'


Spark version:  2.3.0
SparkSQL sesssion created


In [2]:
from pyspark.sql.types import *
from pyspark.sql.functions import col, count, rand, collect_list, explode, struct, count, lit
from pyspark.sql.functions import pandas_udf, PandasUDFType

df = sqlContext.range(0, 10 * 1000 * 1000).withColumn('id', (col('id') / 10000).cast('integer')).withColumn('v', rand())
df2 = df.withColumn('y', rand()).withColumn('x1', rand()).withColumn('x2', rand()).select('id', 'y', 'x1', 'x2')
df2.show()  

+---+--------------------+--------------------+-------------------+
| id|                   y|                  x1|                 x2|
+---+--------------------+--------------------+-------------------+
|  0|  0.5526946702795195| 0.06701956135947407| 0.5924124465081303|
|  0|  0.2913825766137532|  0.1947752518258148| 0.5233665480161437|
|  0|   0.205619978780051|  0.3105616690156754| 0.4916693957807815|
|  0|  0.5266188227740116|  0.7227956133537081| 0.4946251882934882|
|  0|  0.6530891271921427|  0.0150915710311611|0.43486765315462395|
|  0|  0.7923524629572337|  0.4200145687282614| 0.5831081101165458|
|  0|  0.1514894387983231|  0.8493762419095203| 0.7473003496965076|
|  0|  0.7884623552080869|  0.2938184669893035| 0.9341472667774522|
|  0|  0.2519451056797064|0.028705848389948163| 0.5873681776918781|
|  0|  0.4530339702738093| 0.22879789220715907| 0.5728419692926012|
|  0|  0.5691940693721571|  0.7800313534707806| 0.5507994878646254|
|  0|  0.6386162892226206|  0.6837253934633208| 

In [3]:
import pandas as pd
import matplotlib
matplotlib.use('Agg')
import statsmodels.api as sm
# df has four columns: id, y, x1, x2

group_column = 'id'
y_column = 'y'
x_columns = ['x1', 'x2']
schema = df2.select(group_column, *x_columns).schema

@pandas_udf(schema, PandasUDFType.GROUPED_MAP)
# Input/output are both a pandas.DataFrame
def ols(pdf):
    group_key = pdf[group_column].iloc[0]
    y = pdf[y_column]
    X = pdf[x_columns]
    X = sm.add_constant(X)
    model = sm.OLS(y, X).fit()
    return pd.DataFrame([[group_key] + [model.params[i] for i in   x_columns]], columns=[group_column] + x_columns)

beta = df2.groupby(group_column).apply(ols)
beta.show()

Py4JJavaError: An error occurred while calling o170.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 2.0 failed 4 times, most recent failure: Lost task 0.3 in stage 2.0 (TID 6, 10.64.22.220, executor 3): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/opt/spark-2.3.0-bin-hadoop2.7/python/lib/pyspark.zip/pyspark/worker.py", line 218, in main
    func, profiler, deserializer, serializer = read_udfs(pickleSer, infile, eval_type)
  File "/opt/spark-2.3.0-bin-hadoop2.7/python/lib/pyspark.zip/pyspark/worker.py", line 138, in read_udfs
    arg_offsets, udf = read_single_udf(pickleSer, infile, eval_type)
  File "/opt/spark-2.3.0-bin-hadoop2.7/python/lib/pyspark.zip/pyspark/worker.py", line 118, in read_single_udf
    f, return_type = read_command(pickleSer, infile)
  File "/opt/spark-2.3.0-bin-hadoop2.7/python/lib/pyspark.zip/pyspark/worker.py", line 58, in read_command
    command = serializer._read_with_